# CNN MRP Model

This notebook is used to train a CNN model for All class classification on top of the stacking images.

### Importing Required Libraries

This section will import the required libaries that will be used to actually implement the training for the Vanilla RNN Binary Classification Model

In [1]:
import numpy as np
import pandas as pd
import os
import glob
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time

C:\Users\Saksham Goel\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Adding this so that can use the python scripts for loading the training data

In [2]:
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

Importing the python script training data loader function. This function loads the data from the *.wav* files directly

In [3]:
from py_scripts.directory_funcs import *
from py_scripts.wav_file_funcs import *
from py_scripts.misc_audio_signal_funcs import *
from py_scripts.raw_training_data_creation import load_irmas_data

### Loading Training Data

This section defines the training data generators the script from the training data creation module to load the training data.

In [4]:
# First define the training data directory, and find the different classes that exist
training_data_dir = '../../data/mrp_jpg/train/'
validation_data_dir = '../../data/mrp_jpg/validation/'
test_data_dir = '../../data/mrp_jpg/test/'
get_subdirectory_names(training_data_dir)

['gel', 'pia', 'sax', 'vio', 'voi']

Define the generators for each category (Train, Validation, Test)

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../../data/mrp_jpg/train/', target_size=(150, 150),
                                                    batch_size=64, class_mode='categorical')
validaton_generator = validation_datagen.flow_from_directory('../../data/mrp_jpg/validation/', target_size=(150, 150),
                                                             batch_size=64, class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../../data/mrp_jpg/test/', target_size=(150, 150),
                                                  batch_size=64, class_mode='categorical')

Found 1493 images belonging to 5 classes.
Found 318 images belonging to 5 classes.
Found 318 images belonging to 5 classes.


# Defining the Model

In this section, we define the CNN model for the stacked images

In [11]:
model = Sequential()
model.add(Conv2D(16, (5, 5), strides=(2, 2), activation='relu', input_shape = (150, 150, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (3, 3), strides = (1, 1), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (3, 3), strides = (1, 1), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (3, 3), strides = (1, 1), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(5, activation = 'softmax'))
# Compiling the model
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 73, 73, 16)        1216      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 15, 15, 32)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 5, 5, 32)          9248      
__________

In [12]:
# Adding a checkpoint
parent_weight_save_dir = '../../data/Training Results/CNN_MRP/Weights'
tensor_board_dir_path = '../../data/Training Results/CNN_MRP/TensorBoard'
check_output_directory(parent_weight_save_dir)
current_experiment_name = f'CT-{time.time()}'
weight_file_path = os.path.join(parent_weight_save_dir, f'{current_experiment_name}.hdf5')
tensor_board_file_path = os.path.join(tensor_board_dir_path, current_experiment_name)
check_output_directory(tensor_board_file_path)
checkpoint = ModelCheckpoint(weight_file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir=tensor_board_file_path)
early_stopping_criteria = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
callbacks_list = [tensorboard, checkpoint, early_stopping_criteria]

In [13]:
history = model.fit_generator(train_generator, epochs=50, validation_data=validaton_generator, callbacks=callbacks_list)

Epoch 1/50
24/24 [==============================] - 26s 1s/step - loss: 1.4823 - acc: 0.3205 - val_loss: 1.3793 - val_acc: 0.3365

Epoch 00001: val_acc improved from -inf to 0.33648, saving model to ../../data/Training Results/CNN_MRP/Weights\CT-1544943488.2030275.hdf5
Epoch 2/50
24/24 [==============================] - 31s 1s/step - loss: 1.4032 - acc: 0.4137 - val_loss: 1.3238 - val_acc: 0.4403

Epoch 00002: val_acc improved from 0.33648 to 0.44025, saving model to ../../data/Training Results/CNN_MRP/Weights\CT-1544943488.2030275.hdf5
Epoch 3/50
24/24 [==============================] - 25s 1s/step - loss: 1.3600 - acc: 0.4424 - val_loss: 1.2977 - val_acc: 0.4969

Epoch 00003: val_acc improved from 0.44025 to 0.49686, saving model to ../../data/Training Results/CNN_MRP/Weights\CT-1544943488.2030275.hdf5
Epoch 4/50
24/24 [==============================] - 24s 1s/step - loss: 1.2770 - acc: 0.4475 - val_loss: 1.1621 - val_acc: 0.4969

Epoch 00004: val_acc did not improve from 0.49686
Epo